__Lab 03 - Acquisition__

Student Names: [insert your full name(s) here]

Group Number: [insert your group number as in Canvas here]

# Setup

## FPGA Cofigurations & Import Python Libraries

These are the FPGA configurations need when programming it with the bitstream.

BITSTREAM_FILENAME is the path and filename to the Bitstream to program the FPGA with.

Also, we import the Python libraries here.

In [1]:
import numpy as np
import math
import serial
import serial.tools.list_ports
from tqdm import tnrange

DEVICE_NAME = "CmodS7"
DEVICE_ID   = 0
# BITSTREAM_FILENAME = "../Hardware/RO_LFSR_TRNG.bit"
BITSTREAM_FILENAME = "../Hardware/GARO_TRNG.bit"

print("✔️ OK to continue!")

✔️ OK to continue!


## Connect to FPGA

We need to make sure the FPGA is connected.

"%%bash" or "!" indicated that the following code in that cell will run with "bash" instead of python.

In [2]:
!djtgcfg enum

print("✔️ OK to continue!")

Found 1 device(s)

Device: CmodS7
    Device Transport Type: 00020001 (USB)
    Product Name:          Digilent Cmod S7
    User Name:             CmodS7
    Serial Number:         210376AFE086
✔️ OK to continue!


Make sure to that it says "Found 1 device(s)" (or more) and the device name match with "DEVICE_NAME" from section 1.1

Also, we can verify the device ID (DEVICE_ID) from section 1.1 with the following code.

(DEVICE_ID should be 0 for CmodS7, unless you have other FPGA connected)

In [3]:
%%bash -s "$DEVICE_NAME"
djtgcfg init -d $1

Initializing scan chain...
Found Device ID: 037c4093

Found 1 device(s):
    Device 0: XC7S25


The actual device ID is located in the list at the bottom. It should say, for example:

<span class="mark">Found 1 device(s):</span> <br>
<pre> <span class="mark">Device 0: XC7S25</span>

The 0 here is the Device ID for "XC7S25" (i.e., CmodS7).

## Program the FPGA

Program the FPGA with the bitstream.

Also, make sure the BITSTREAM_FILENAME in section 1.1 matches with bitstream you want to programm the FPGA with.

In [4]:
%%bash -s "$DEVICE_NAME" "$DEVICE_ID" "$BITSTREAM_FILENAME"
djtgcfg prog -d $1 -i $2 -f $3

Programming device. Do not touch your board. This may take a few minutes...
Programming succeeded.


## Setup the USB serial communication port

Now, we can setup PySerial.

First, let's identify the USB port name where the FPGA is connected to. Use the function below to print a list of all your ports.

In [5]:
def print_ports():
    portsList = serial.tools.list_ports.comports()

    print("--- PORTS LIST ---")
    for p in sorted(portsList):
        # first prints the port, then the device product
        print(f'{p[0]} : {p[1]}')

print_ports()

print("✔️ OK to continue!")

--- PORTS LIST ---
/dev/ttyUSB1 : Digilent Adept USB Device - Digilent Adept USB Device
✔️ OK to continue!


In [6]:
# connect to COM port, set baudrate
port = "/dev/ttyUSB1"
boudrate = 115200
ser = serial.Serial(port, boudrate)

print("✔️ OK to continue!")

✔️ OK to continue!


# Send & Collect Data

Send and collect the data from the FPGA as requested in the lab manual. Make sure to format your data correctly.

Note: your final collected data should in `bytearray` (or `bytes`) type (i.e., raw bytes)

In [7]:
# Function: get_1resp()
#   Input: 'ser' - serial connection to FPGA
#   Input: 'in_hex' - an input hex string to send to the FPGA
#   Input: 'resp_size' - size of the output response in bytes
#   Output: 'r_bin' - respective binary string response of size 'resp_size' bytes (8*resp_size bits).

def get_resp(ser: serial.Serial, num_responses: int) -> str:

    resp_size = 1 # read one byte at a time
    number_of_responses = num_responses
    in_bytes = num_responses.to_bytes(3, 'big')
    

    

    # send bytes to FPGA
    ser.write(in_bytes)
    
    while ser.in_waiting < 1:
        pass
        
    r_bytes = ser.read(1)
    read_count = 1

    while (True):
        if ser.in_waiting >= 1:
            r_bytes = r_bytes + ser.read(1)
            read_count = read_count + 1
        if ser.in_waiting < 1 and read_count == number_of_responses:
            print("All bytes read.")
            break
            
    
    return r_bytes
        
     

print("✔️ OK to continue!")

✔️ OK to continue!


In [8]:
random_bytes = get_resp(ser, 1250000)
print(len(random_bytes))


All bytes read.
1250000


# Save Data

Save your __raw bytes__ into a file. `"wb"` lets you write to the file with raw bytes.

In [9]:
# Open file to write raw bytes
filename = "../Data/GARO_10M.bin"
f = open(filename, "wb")
f.write(random_bytes)
# Remember to close the file at the end
f.close()